In [3]:
import librosa
import random
import pandas as pd
import numpy as np
import os
import librosa

import pickle

from tqdm.auto import tqdm

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings(action='ignore') 

import argparse
import torch
import torch.nn as nn

In [4]:
%cd /workspace/EmotionShortForm/

/workspace/EmotionShortForm


## Video to Wav

In [8]:
from moviepy.editor import *

video_path = 'video_data/[#유퀴즈온더블럭] MBTI 검사 같은 거 안 하는 서지니 형의 헤어나올 수 없는 매력😉 청문회 방불케 하는 츤데레의 인간화, 이서진 탐구 생활! _ #지금꼭볼동영상.mp4'

audio_path = 'audio_data/audio2.wav'  # 추출할 오디오 파일 경로

# Load movie file as VideoFileClip object
clip = VideoFileClip(video_path)

# audio extraction
clip.audio.write_audiofile(audio_path)

# 객체 삭제
clip.close()

chunk:   2%|▏         | 355/22798 [00:00<00:06, 3509.97it/s, now=None]

MoviePy - Writing audio in audio_data/audio2.wav


MoviePy - Done.


In [9]:
import IPython
IPython.display.Audio('audio_data/audio2.wav')

## Trim Audio 2 sec

In [13]:
# wav 파일 불러오기
audio_file = 'audio_data/audio2.wav'
y, sr = librosa.load(audio_file)

# define trim audio function
def trim_audio_2_sec(y,sr):
    # 2 sec 단위로 자르기
    interval = 2 # 2 sec
    sample_rate = sr * interval 
    audio_list = []
    for i in tqdm(range(0, len(y), sample_rate)):
        audio_list.append(y[i:i+sample_rate])
    return audio_list

audio_list = trim_audio_2_sec(y, sr)

## Preprocessing

In [14]:
CFG = {
    'SR':16000,
    'N_MFCC':32, # MFCC coefficient
    'SEED':41
}

In [16]:
def preprocessing(audio_list, sr):
    features = []
    
    for audio in tqdm(audio_list):
        # mfcc
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])
 
        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in mfcc:
            y_feature.append(np.mean(e))
        features.append(y_feature)
        
#     mel_df = pd.DataFrame(features, columns=['mfcc_'+ str(x) for x in range(1,CFG['N_melspectrogram']+1)])
    return features

In [18]:
features = preprocessing(audio_list,sr)

KeyError: 'N_melspectrogram'

### Get MFCC data frame

In [19]:
mfcc_df = pd.DataFrame(features, columns=['mfcc_'+ str(x) for x in range(1,CFG['N_MFCC']+1)])

In [21]:
mfcc_df

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,...,mfcc_23,mfcc_24,mfcc_25,mfcc_26,mfcc_27,mfcc_28,mfcc_29,mfcc_30,mfcc_31,mfcc_32
0,-286.700317,85.89801,0.010341,9.878551,-14.822516,-8.270623,-11.34886,-12.601942,-15.246858,-2.847366,...,-5.54776,-3.748027,-3.769446,-1.98018,-3.236493,-0.577646,-2.656336,-0.195218,-2.301059,1.080843
1,-286.700317,85.89801,0.010341,9.878551,-14.822516,-8.270623,-11.34886,-12.601942,-15.246858,-2.847366,...,-5.54776,-3.748027,-3.769446,-1.98018,-3.236493,-0.577646,-2.656336,-0.195218,-2.301059,1.080843
2,-286.700317,85.89801,0.010341,9.878551,-14.822516,-8.270623,-11.34886,-12.601942,-15.246858,-2.847366,...,-5.54776,-3.748027,-3.769446,-1.98018,-3.236493,-0.577646,-2.656336,-0.195218,-2.301059,1.080843
3,-286.700317,85.89801,0.010341,9.878551,-14.822516,-8.270623,-11.34886,-12.601942,-15.246858,-2.847366,...,-5.54776,-3.748027,-3.769446,-1.98018,-3.236493,-0.577646,-2.656336,-0.195218,-2.301059,1.080843
4,-286.700317,85.89801,0.010341,9.878551,-14.822516,-8.270623,-11.34886,-12.601942,-15.246858,-2.847366,...,-5.54776,-3.748027,-3.769446,-1.98018,-3.236493,-0.577646,-2.656336,-0.195218,-2.301059,1.080843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,-286.700317,85.89801,0.010341,9.878551,-14.822516,-8.270623,-11.34886,-12.601942,-15.246858,-2.847366,...,-5.54776,-3.748027,-3.769446,-1.98018,-3.236493,-0.577646,-2.656336,-0.195218,-2.301059,1.080843
513,-286.700317,85.89801,0.010341,9.878551,-14.822516,-8.270623,-11.34886,-12.601942,-15.246858,-2.847366,...,-5.54776,-3.748027,-3.769446,-1.98018,-3.236493,-0.577646,-2.656336,-0.195218,-2.301059,1.080843
514,-286.700317,85.89801,0.010341,9.878551,-14.822516,-8.270623,-11.34886,-12.601942,-15.246858,-2.847366,...,-5.54776,-3.748027,-3.769446,-1.98018,-3.236493,-0.577646,-2.656336,-0.195218,-2.301059,1.080843
515,-286.700317,85.89801,0.010341,9.878551,-14.822516,-8.270623,-11.34886,-12.601942,-15.246858,-2.847366,...,-5.54776,-3.748027,-3.769446,-1.98018,-3.236493,-0.577646,-2.656336,-0.195218,-2.301059,1.080843


## Model Load

### Load emotion classification model

In [24]:
import torch
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, (hidden_state, cell_state) = self.lstm(x, (h0, c0))
        output = self.fc(out[:,-1,:])
        hidden = self.fc(hidden_state[-1])
        return output, hidden,  out[:,-1,:]

In [30]:
input_size = 32
hidden_size = 128
num_layers=3
output_size=7

In [36]:
# load model

model = LSTM(input_size, hidden_size, num_layers, output_size)
model.load_state_dict(torch.load('./model/lstm_emotion_classification_model.pt'))

<All keys matched successfully>

In [37]:
criterion = nn.CrossEntropyLoss() # CrossEntropyLoss, BCEWithLogitsLoss
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5)

In [57]:
test_emotion = torch.from_numpy(mfcc_df.to_numpy()).float()

In [58]:
test_emotion.shape

torch.Size([517, 32])

In [63]:
with torch.no_grad():
    model.eval()
    outputs,hidden,out = model(test_emotion.unsqueeze(1))
    emotion_lstm_feature = out
    print(f'emotion_lstm_feature:{emotion_lstm_feature}') 
    print(emotion_lstm_feature.shape) ## 128 size

emotion_lstm_feature:tensor([[ 0.0902, -0.1575, -0.4638,  ...,  0.3098,  0.2971,  0.3810],
        [ 0.0902, -0.1575, -0.4638,  ...,  0.3098,  0.2971,  0.3810],
        [ 0.0902, -0.1575, -0.4638,  ...,  0.3098,  0.2971,  0.3810],
        ...,
        [ 0.0902, -0.1575, -0.4638,  ...,  0.3098,  0.2971,  0.3810],
        [ 0.0902, -0.1575, -0.4638,  ...,  0.3098,  0.2971,  0.3810],
        [ 0.0902, -0.1575, -0.4638,  ...,  0.3098,  0.2971,  0.3810]])
torch.Size([517, 128])


### Load arousal model

In [52]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, (hidden_state, cell_state) = self.lstm(x, (h0, c0))
        output = self.fc(out[:,-1,:])
        hidden = self.fc(hidden_state[-1])
        return output, hidden,  out[:,-1,:]

In [54]:
input_size = 32
hidden_size =  64
num_layers = 3
output_size = 1
learning_rate = 0.001

In [56]:
# load model
model_arousal = LSTM(input_size, hidden_size, num_layers, output_size)
model_arousal.load_state_dict(torch.load('./model/lstm_arousal_model.pt'))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_arousal.parameters(), lr=learning_rate)


In [61]:
test_arousal = torch.from_numpy(mfcc_df.to_numpy()).float()
test_arousal.shape

torch.Size([517, 32])

In [67]:
with torch.no_grad():
    model_arousal.eval()
    outputs,hidden,out = model_arousal(test_arousal.unsqueeze(1))
    arousal_lstm_feature = out
    print(f'arousal_lstm_feature:{arousal_lstm_feature}') 
    print(arousal_lstm_feature.shape) ## 64 size

arousal_lstm_feature:tensor([[ 0.5654, -0.5263,  0.5208,  ..., -0.4331,  0.4324, -0.5674],
        [ 0.5654, -0.5263,  0.5208,  ..., -0.4331,  0.4324, -0.5674],
        [ 0.5654, -0.5263,  0.5208,  ..., -0.4331,  0.4324, -0.5674],
        ...,
        [ 0.5654, -0.5263,  0.5208,  ..., -0.4331,  0.4324, -0.5674],
        [ 0.5654, -0.5263,  0.5208,  ..., -0.4331,  0.4324, -0.5674],
        [ 0.5654, -0.5263,  0.5208,  ..., -0.4331,  0.4324, -0.5674]])
torch.Size([517, 64])


### Load valence model

In [70]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, (hidden_state, cell_state) = self.lstm(x, (h0, c0))
        output = self.fc(out[:,-1,:])
        hidden = self.fc(hidden_state[-1])
        return output, hidden,  out[:,-1,:]

In [71]:
input_size = 32
hidden_size = 32 # 32
num_layers = 4 # 2
output_size = 1
learning_rate = 0.001
num_epochs = 100 # 200

In [72]:
# load model
model_valence = LSTM(input_size, hidden_size, num_layers, output_size)
model_valence.load_state_dict(torch.load('./model/lstm_valence_model.pt'))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_valence.parameters(), lr=learning_rate)


In [73]:
test_valence = torch.from_numpy(mfcc_df.to_numpy()).float()
test_valence.shape

torch.Size([517, 32])

In [74]:
with torch.no_grad():
    model_valence.eval()
    outputs,hidden,out = model_valence(test_valence.unsqueeze(1))
    valence_lstm_feature = out
    print(f'valence_lstm_feature:{valence_lstm_feature}') 
    print(valence_lstm_feature.shape) ## 32 size

valence_lstm_feature:tensor([[-0.3568, -0.2716,  0.0326,  ...,  0.5994,  0.5142, -0.6592],
        [-0.3568, -0.2716,  0.0326,  ...,  0.5994,  0.5142, -0.6592],
        [-0.3568, -0.2716,  0.0326,  ...,  0.5994,  0.5142, -0.6592],
        ...,
        [-0.3568, -0.2716,  0.0326,  ...,  0.5994,  0.5142, -0.6592],
        [-0.3568, -0.2716,  0.0326,  ...,  0.5994,  0.5142, -0.6592],
        [-0.3568, -0.2716,  0.0326,  ...,  0.5994,  0.5142, -0.6592]])
torch.Size([517, 32])


## Feature concatenation

In [80]:
import pandas as pd

feature_combined = torch.cat((emotion_lstm_feature, arousal_lstm_feature, valence_lstm_feature), dim=1)

print(feature_combined.shape) # 32 + 64 + 128 = 224
print(feature_combined)

torch.Size([517, 224])
tensor([[ 0.0902, -0.1575, -0.4638,  ...,  0.5994,  0.5142, -0.6592],
        [ 0.0902, -0.1575, -0.4638,  ...,  0.5994,  0.5142, -0.6592],
        [ 0.0902, -0.1575, -0.4638,  ...,  0.5994,  0.5142, -0.6592],
        ...,
        [ 0.0902, -0.1575, -0.4638,  ...,  0.5994,  0.5142, -0.6592],
        [ 0.0902, -0.1575, -0.4638,  ...,  0.5994,  0.5142, -0.6592],
        [ 0.0902, -0.1575, -0.4638,  ...,  0.5994,  0.5142, -0.6592]])
